In [1]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-2x2x2/"

In [2]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [3]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [4]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 16108.70it/s]


In [5]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )
       

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))


        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")


    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 8/8 [00:00<00:00, 139.01it/s]

Standard deviation exceeded 1e-16 tolerance: 2.5494915419483335e-15, for i_q:0


In [6]:
from kernel import KernelMtxEl

q0val = [0.001, 0.001, 0.001]


In [7]:
kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0=q0val,
    parallel=False,
    num_bands_val=4,
    num_bands_con=4,
)

In [8]:
data = kernelclass.kernel_mtxel()

In [9]:
exc = data["exc"]
print(exc.shape)
exc = exc[:, 0, 0, 0, :, 0]

for row in range(exc.shape[0]):
    row_data = []
    for col in range(exc.shape[1]):
        row_data.append(f"{exc[row, col]:.5f}")
    print(" | ".join(row_data))

(8, 8, 4, 4, 4, 4)
0.47881-0.00000j | -0.04249-0.11703j | -0.07744+0.11279j | -0.14912-0.04517j
-0.02994-0.11418j | -0.09566-0.02049j | 0.00000+0.00000j | -0.00923+0.05191j
-0.07084-0.02386j | -0.07431-0.08646j | 0.00000-0.00000j | 0.00429-0.02547j
-0.06933+0.12470j | 0.00074+0.00153j | 0.19405+0.28543j | 0.00221+0.00358j
-0.10282+0.15727j | -0.01911+0.00103j | -0.00000-0.00000j | 0.10199+0.05572j
0.01616-0.05548j | 0.00116-0.00065j | -0.03015-0.13417j | -0.00258-0.00320j
0.02306+0.09088j | -0.00104+0.00002j | 0.21673-0.04781j | 0.00241-0.00823j
0.00050-0.00007j | -0.00079+0.00097j | 0.00000-0.00000j | -0.00624-0.02131j


In [10]:
head = data["head"]
head = head[:, 0, 0, 0, :, 0]

for row in range(head.shape[0]):
    row_data = []
    for col in range(head.shape[1]):
        row_data.append(f"{head[row, col]:.5f}")
    print(" | ".join(row_data))

1.00000-0.00000j | -0.00000+0.00000j | -0.00000+0.00000j | 0.00000-0.00000j
-0.05016-0.19131j | -0.00125-0.00027j | -0.04869+0.00027j | -0.00623+0.03513j
-0.11326-0.03814j | 0.00013+0.00015j | 0.02803-0.00070j | 0.00341-0.02026j
-0.12329+0.21733j | -0.00113+0.00381j | 0.06187+0.09459j | 0.09721-0.05767j
-0.17991+0.27518j | -0.00256+0.00014j | 0.08066+0.00778j | 0.05211+0.02848j
0.02843-0.09821j | 0.00201-0.00116j | -0.00912-0.04691j | -0.03805+0.02892j
0.04308+0.15990j | -0.00157+0.00185j | 0.07366-0.01392j | -0.07127-0.02325j
-0.00000+0.00000j | -0.00008+0.00010j | 0.00884-0.00033j | -0.00182-0.00622j


In [11]:
wings = data["wings"]
wings = wings[:, 0, 0, 0, :, 0]

for row in range(wings.shape[0]):
    row_data = []
    for col in range(wings.shape[1]):
        row_data.append(f"{wings[row, col]:.5f}")
    print(" | ".join(row_data))

0.00000+0.00000j | 0.00000+0.00000j | 0.00000+0.00000j | 0.00000+0.00000j
-0.02333-0.08898j | -0.00058-0.00013j | 0.00859-0.00005j | -0.00316+0.01779j
-0.05267-0.01774j | 0.00006+0.00007j | -0.00495+0.00012j | 0.00173-0.01026j
0.00336-0.00592j | 0.00003-0.00010j | -0.00229-0.00349j | -0.00359+0.00212j
-0.08366+0.12798j | -0.00119+0.00007j | -0.01424-0.00137j | 0.02639+0.01442j
-0.00077+0.00268j | -0.00005+0.00003j | 0.00034+0.00173j | 0.00140-0.00107j
-0.00117-0.00436j | 0.00004-0.00005j | -0.00272+0.00052j | 0.00263+0.00086j
-0.00000+0.00000j | -0.00007+0.00008j | -0.00270+0.00010j | -0.00160-0.00546j


In [12]:
body = data["body"]
body = body[:, 0, 0, 0, :, 0]

for row in range(body.shape[0]):
    row_data = []
    for col in range(body.shape[1]):
        row_data.append(f"{body[row, col]:.5f}")
    print(" | ".join(row_data))

0.26817-0.00000j | -0.02549-0.07022j | -0.04373+0.06368j | -0.11062-0.03350j
-0.08153-0.31100j | 0.05704+0.01221j | 0.06070-0.00033j | -0.00875+0.04920j
-0.18194-0.06128j | 0.04663+0.05426j | -0.03495+0.00086j | 0.00415-0.02460j
-0.20591+0.37437j | 0.00442+0.00351j | 0.21275+0.30972j | -0.08405+0.05822j
-0.29583+0.45252j | 0.00681-0.00036j | -0.10056-0.00970j | 0.09405+0.05139j
0.04851-0.16597j | 0.00348-0.00192j | -0.03392-0.14523j | 0.03083-0.03054j
0.06688+0.27254j | -0.00317-0.00162j | 0.23370-0.05366j | 0.06738+0.01084j
0.01449-0.00175j | -0.00377+0.00461j | -0.01102+0.00040j | -0.00554-0.01893j


In [13]:
exc_QTM = data["exc"]
head_QTM = data["head"]
wings_QTM = data["wings"]
body_QTM = data["body"]

print(exc_QTM.shape)
print(head_QTM.shape)
print(wings_QTM.shape)
print(body_QTM.shape)

(8, 8, 4, 4, 4, 4)
(8, 8, 4, 4, 4, 4)
(8, 8, 4, 4, 4, 4)
(8, 8, 4, 4, 4, 4)


In [14]:
with h5py.File(dirname + 'bsemat.h5', 'r') as f:
    body_BGW = f['mats/body']
    head_BGW = f['mats/head']
    wings_BGW = f['mats/wing']
    exc_BGW = f['mats/exchange']

    body_BGW = body_BGW[..., 0] + body_BGW[..., 1] * 1j
    head_BGW = head_BGW[..., 0] + head_BGW[..., 1] * 1j
    wings_BGW = wings_BGW[..., 0] + wings_BGW[..., 1] * 1j
    exc_BGW = exc_BGW[..., 0] + exc_BGW[..., 1] * 1j

    print("Body_BGW", body_BGW.shape)
    print("Head_BGW", head_BGW.shape)
    print("Wings_BGW", wings_BGW.shape)
    print("Exc_BGW", exc_BGW.shape)

Body_BGW (8, 8, 4, 4, 4, 4)
Head_BGW (8, 8, 4, 4, 4, 4)
Wings_BGW (8, 8, 4, 4, 4, 4)
Exc_BGW (8, 8, 4, 4, 4, 4)


In [15]:
exc_BGW = exc_BGW.transpose(1, 0, 3, 2, 5, 4)
body_BGW = body_BGW.transpose(1, 0, 3, 2, 5, 4)
head_BGW = head_BGW.transpose(1, 0, 3, 2, 5, 4)
wings_BGW = wings_BGW.transpose(1, 0, 3, 2, 5, 4)

In [16]:
diff_exc = np.abs(exc_BGW - exc_QTM)
sum_exc = diff_exc.sum(axis=(2, 3, 4, 5))
norm_diff_exc = sum_exc / (4 * 4 * 4 * 4)

In [17]:
diff_body = np.abs(body_BGW - body_QTM)
sum_body = diff_body.sum(axis=(2, 3, 4, 5))
norm_diff_body = sum_body / (4 * 4 * 4 * 4)

In [18]:
diff_head = np.abs(head_BGW - head_QTM)
sum_head = diff_head.sum(axis=(2, 3, 4, 5))
norm_diff_head = sum_head / (4 * 4 * 4 * 4)

In [19]:
diff_wings = np.abs(wings_BGW - wings_QTM)
sum_wings = diff_wings.sum(axis=(2, 3, 4, 5))
norm_diff_wings = sum_wings / (4 * 4 * 4 * 4)